# Personal Financial Report
>version 3.0

this jupyter notebook allows me to keep track of my finances.









# SET UP

In [1]:
## import the Libraries

# standard Libraries
import os
import re
import time
import json5 as json 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import functions as f

# used for displaying data and stuff
from IPython.display import display, HTML, FileLink, FileLinks

# used for charts and Graphs
import plotly
import plotly.express as px
import seaborn as sns
import calmap
from plotly_calplot import calplot

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# plotly Theme
plotly_theme = 'plotly_dark'

# disable plotly warnings
import warnings
warnings.filterwarnings("ignore")

## Pandas Options

# show all the columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

# turn off a warning
pd.options.mode.chained_assignment = None  # default='warn'

## Variables
DIR = os.getcwd()
data_DIR = os.path.join(DIR,'data')
old_data_DIR = os.path.join(DIR,'old_data')

if os.path.exists(data_DIR) == False:
    os.mkdir(data_DIR)


In [32]:
this_year = int(datetime.now().strftime("%Y"))
this_month = int(datetime.now().strftime("%m"))

this_year = 2022

# incorporated new data

In [11]:
# merges the old data with the new data...
# and opens the file so the user can fill in the spots that are not automatically filled


f.incorporated_data(autofillcat= False)
f.open_year_in_excel(this_year)

# f.open_year_in_scalc(this_year)

Happy New Years!
[Errno 2] No such file or directory: 'C:\\Users\\JGarza\\Downloads\\stmt.csv'
error loading new data


# create dataframes

df_year  
df_90days  
df_30days  
df_07days  
~~df_all~~  


In [33]:
# df_year = f.load_last_x_years(2)
df_year = f.load_year(this_year)

df_year.Date = pd.to_datetime(df_year.Date)

ninety_days = ( df_year.Date.max() - timedelta(days=90)) 
df_90days = df_year[ df_year['Date'] >= ninety_days]
# display(df_90days)

thirty_days = ( df_year.Date.max() - timedelta(days=30)) 
df_30days = df_year[ df_year['Date'] >= thirty_days]
# display(df_30days)

seven_days = ( df_year.Date.max() - timedelta(days=7)) 
df_07days = df_year[ df_year['Date'] >= seven_days]
# display(df_07days)

# df_all = f.get_all_data()


In [34]:
display(HTML('<h2>Date Range</h2>'))
display(HTML(f'<p>{df_year["Date"].min()} to {df_year["Date"].max()}</p>'))


# 🌞 SunBurst

In [35]:
# print(*px.colors.named_colorscales())

In [36]:
def make_sunburst(df,title):
    display(HTML('<h2> 🌞 SunBurst: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))

    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(temp['Delta'].sum())))

    fig = px.sunburst(
        temp,
        # path=['YYYYMM','Category','Location'],
        path=['Category','Location'],
        values='Delta',
        color='Delta', 
        hover_data=['Category','Location','Delta','YYYYMMDD'],
        color_continuous_scale='rdylgn_r',
        color_continuous_midpoint=np.average(temp['Delta'], weights=temp['Delta']),
        template = plotly_theme,
        height=600
    )
    fig.show()


In [37]:
make_sunburst(df_07days,'7 days')
make_sunburst(df_30days,'30 days')
make_sunburst(df_90days,'90 days')
make_sunburst(df_year,'Year')

# 🔥 HeatMaps

In [38]:
## run this for a list of the colorscales
# print(*px.colors.named_colorscales())

def heatmap_time_cat(df, time_column, max_value = None, title = '',colorscale = 'RdYlGn'):
    """
    creates a heatmap with time and category 
    df: dataframe
    time_column: the column to use for the columns
    max_value: the value that will reprement the max color in the color scale 
        * min_value = max_value * -1
    title: the title to display 
    colorscale: the columns to use
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp = df.copy()
    temp[time_column] = temp[time_column].astype(float).astype(str)
    
    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = 'Category',
        columns = time_column,
        aggfunc= {'Delta':sum,}
        )
    
    temp = temp.fillna(0)
    temp = temp.reset_index()
    temp = temp[temp.columns.tolist()]
    
    if 'nan' in temp.columns.tolist():
        temp = temp.drop(columns=['nan'])
    
    cols = temp.columns.tolist()
    cols = [c for c in cols if c != 'Category']
    
    
    # Sum of Rows
    temp['Σ'] = temp[cols].sum(axis=1)

    # sort by row sum
    temp = temp.sort_values(by='Σ',ascending=False)
    
    # Sum of Cols
    sums = {}
    absminmax = 0
    for c in temp.columns:
        if c == 'Category':
            sums['Category'] = 'GrandTotal'
        else:
            sums[c] = temp[c].sum()
            absminmax = max([absminmax,abs(sums[c])])
    # print(sums)
    sums = pd.DataFrame(sums,index=[99])

    temp = pd.concat([temp,sums])
    temp = temp.reset_index(drop=True)
    
    cm = sns.color_palette(colorscale, as_cmap=True)
    
    if max_value == None:
        display(temp.style.background_gradient(cmap=cm, vmin=absminmax*-1, vmax=absminmax).format(precision=2,thousands=','))
    else:
        display(temp.style.background_gradient(cmap=cm, vmin=max_value*-1, vmax=max_value).format(precision=2,thousands=','))
    

def heatmap2_year_view(df, title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2'))
    
    temp = df.copy()

    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = 'Date',
        aggfunc= {'Delta':sum,}
        )
    temp = temp.reset_index(drop=False)
    
    events = pd.Series(  list(temp['Delta']) , list(temp['Date']) )
    calmap.calendarplot(events,cmap='RdYlGn', daylabels='MTWTFSS')
    
def heatmap3_year_view(df,title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2>'))
    
    temp = df.copy()
    
    # get the day of the week
    temp['DOW_'] = pd.to_datetime(temp['Date']).dt.strftime("%w")
    temp['DOW'] = pd.to_datetime(temp['Date']).dt.strftime("%a")
    temp['M.W'] = pd.to_datetime(temp['Date']).dt.strftime("%m.%W")

    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = ['DOW_','DOW'],
        columns = 'M.W',
        aggfunc= {
            'Delta':sum,
            }
        )
    
    temp = temp.reset_index()
    temp = temp.drop(columns=['DOW_'])
    temp.set_index("DOW", inplace=True)
    
    temp = temp.fillna(0)
    
    fig = px.imshow(
        temp, 
        text_auto=True, 
        # aspect="auto",
        color_continuous_scale='rdylgn',
        color_continuous_midpoint = 0.0,
        template = plotly_theme,
        height=500,
        )
    fig.update_xaxes(side="top")
    fig.show()

def heatmap4_year_view(df,title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 

    --this doesn't work as well as the other one
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2>'))

    temp = df.copy()
    temp = temp.fillna(0.0)

    fig = calplot(
        temp,
        x="Date",
        y="Delta",
        dark_theme=True,
        colorscale="rdylgn",
        total_height=600
    )
    fig.show()

    

In [39]:
# heatmap_time_cat(df_07days,time_column='YYYYMMDD',max_value=500,title='7 days')
heatmap_time_cat(df_07days,time_column='YYYYMMDD',max_value=1000,title='7 days')

,Category,20220915.0,20220916.0,20220919.0,20220920.0,20220921.0,20220922.0,Σ
0,invest,0.00,0.00,"1,261.00",853.00,0.00,-904.00,"1,210.00"
1,paypal,0.00,943.00,0.00,0.00,0.00,0.00,943.00
2,debt,0.00,0.00,829.00,0.00,0.00,0.00,829.00
3,tech,0.00,158.00,0.00,0.00,488.00,0.00,646.00
4,clothing,0.00,0.00,950.00,-818.00,0.00,0.00,132.00
5,misc,0.00,0.00,-3.00,0.00,0.00,0.00,-3.00
6,grocery,0.00,0.00,-168.00,0.00,0.00,0.00,-168.00
7,unk,0.00,0.00,20.00,0.00,0.00,-570.00,-550.00
8,utility,-881.00,0.00,0.00,0.00,0.00,0.00,-881.00
9,health,-573.00,-86.00,0.00,387.00,0.00,"-1,146.00","-1,418.00"


In [40]:
# heatmap_time_cat(df_30days,time_column='YYYY.W',max_value=500,title='30 days')
heatmap_time_cat(df_30days,time_column='YYYY.W',max_value=1000,title='30 days')

,Category,2022.34,2022.35,2022.36,2022.37,2022.38,Σ
0,debt,-30.00,-231.00,677.00,0.00,829.00,"1,245.00"
1,health,603.00,214.00,530.00,242.00,-759.00,830.00
2,utility,0.00,238.00,828.00,-524.00,0.00,542.00
3,clothing,0.00,23.00,-996.00,"1,217.00",132.00,376.00
4,tech,-368.00,230.00,"-1,430.00",510.00,488.00,-570.00
5,unk,"1,406.00",308.00,-927.00,-819.00,-550.00,-582.00
6,misc,0.00,-38.00,-873.00,331.00,-3.00,-583.00
7,paypal,-37.00,-439.00,"-1,596.00",943.00,0.00,"-1,129.00"
8,travel,0.00,-158.00,"1,226.00","-1,316.00","-1,562.00","-1,810.00"
9,invest,-390.00,"-2,256.00",-140.00,"-1,034.00","1,210.00","-2,610.00"


In [41]:
# heatmap_time_cat(df_90days,time_column='YYYYMM',max_value=500,title='90 days')
heatmap_time_cat(df_90days,time_column='YYYYMM',max_value=1000,title='90 days')

,Category,202206.0,202207.0,202208.0,202209.0,Σ
0,misc,0.00,"2,870.00","1,950.00",-583.00,"4,237.00"
1,unk,"-2,368.00","2,430.00","3,412.00","-2,296.00","1,178.00"
2,utility,-895.00,832.00,43.00,542.00,522.00
3,paypal,324.00,42.00,-295.00,-653.00,-582.00
4,tech,-581.00,-756.00,894.00,-432.00,-875.00
5,invest,-557.00,"1,453.00","-2,120.00",36.00,"-1,188.00"
6,debt,0.00,-699.00,"-2,144.00","1,506.00","-1,337.00"
7,health,-250.00,"-1,259.00",-137.00,227.00,"-1,419.00"
8,travel,"1,791.00",-801.00,"-1,217.00","-1,652.00","-1,879.00"
9,clothing,0.00,"-1,146.00","-1,399.00",353.00,"-2,192.00"


In [42]:
# heatmap_time_cat(df_year,time_column='YYYYMM',max_value=500,title='Year')
heatmap_time_cat(df_year,time_column='YYYYMM',max_value=1000,title='Year')

,Category,202201.0,202202.0,202203.0,202204.0,202205.0,202206.0,202207.0,202208.0,202209.0,Σ
0,grocery,"2,046.00","1,604.00","1,103.00","3,411.00","3,306.00",-21.00,"1,643.00","-2,229.00","-2,750.00","8,113.00"
1,misc,36.00,"-1,273.00",708.00,"3,641.00",-465.00,-727.00,"2,870.00","1,950.00",-583.00,"6,157.00"
2,invest,"1,147.00",398.00,"3,838.00","1,249.00","-1,897.00",294.00,"1,453.00","-2,120.00",36.00,"4,398.00"
3,unk,"2,129.00",0.00,"1,607.00","1,476.00","-1,948.00","-4,213.00","2,430.00","3,412.00","-2,296.00","2,597.00"
4,debt,743.00,0.00,162.00,818.00,41.00,548.00,-699.00,"-2,144.00","1,506.00",975.00
5,clothing,71.00,508.00,0.00,97.00,274.00,-139.00,"-1,146.00","-1,399.00",353.00,"-1,381.00"
6,utility,-865.00,0.00,"-1,346.00",800.00,"-1,489.00",-88.00,832.00,43.00,542.00,"-1,571.00"
7,tech,97.00,0.00,961.00,"-2,395.00","-1,253.00",-307.00,-756.00,894.00,-432.00,"-3,191.00"
8,paypal,171.00,0.00,"-1,580.00","-1,336.00",202.00,53.00,42.00,-295.00,-653.00,"-3,396.00"
9,travel,"1,544.00",-455.00,391.00,954.00,"-3,024.00",-591.00,-801.00,"-1,217.00","-1,652.00","-4,851.00"


In [43]:
# not working with pandas 2.0.3
# heatmap2_year_view(df_year,title='Year_alt1')

In [44]:
heatmap3_year_view(df_year,'Year_alt2')

# 🔥🗓️ HeatCalendar

In [45]:
# df_year[ df_year['YYYYMMDD'] == 20231226]['Delta'].sum()

df_year.query(f'YYYYMMDD == {20231226}')['Delta'].sum()


0

In [46]:
import calendar

cmap = plt.get_cmap('RdYlGn')

# get the largest absolute delta per day
temp = pd.pivot_table(
    df_year,
    values = 'Delta',
    index = 'YYYYMMDD',
    aggfunc= {'Delta':sum,}
    )
absmax = max(abs(temp['Delta'].min()) ,temp['Delta'].max())
absmax *= 0.5


for mon_num in range(1,13):

    m = calendar.month(this_year,mon_num)

    # print(m)

    mon_table = ''
    for index,l in enumerate(m.split('\n')):
        if index == 0:
            mon_table += f'<tr><td>{this_year}</td><td>{calendar.month_name[mon_num]}</td></tr>'
        elif index == 1:
            mon_table += f'<tr><td>Mo </td><td>Tu </td><td>We </td><td>Th </td><td>Fr </td><td>Sa </td><td>Su </td></tr>'
        else:

            wstr = ''
            for i in range(0,len(l),3):
                d_sum = 0
                
                try:
                    year_month_day = this_year * 10000 + mon_num * 100 + int(l[i:i+2].replace(' ', ''))
                    d_sum = df_year.query(f'YYYYMMDD == {year_month_day}')['Delta'].sum()
                except:
                    pass

                normalized_value = (d_sum + absmax) / (absmax*2)
                normalized_value = max(0.0, min(normalized_value, 1.0))
                # print(normalized_value)
                color = cmap( normalized_value )
                # print(color)
                r, g, b, _ = color
                

                d = l[i:i+2].replace(' ',' ')+ ''
                wstr += f'<td style="height:1rem; width: 3rem; background-color:rgb({int(r*255)},{int(g*255)},{int(b*255)}); color:black"><b>{d}<br/>{int(d_sum)}</b></td>'
            mon_table += '<tr>' + wstr + '</tr>'
    # print(mon_table)
    display(HTML('<table>' + mon_table + '</table>'))




# 📊 Bar-Chart

In [47]:
def make_barchart(df,time_column,title):
    display(HTML('<h2> 📊 Bar-Chart: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(temp['Delta'].sum())))

    temp[time_column] = temp[time_column].astype(float).astype(str)
    
    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = [time_column,'Category'],
        aggfunc= {'Delta':sum,}
        )

    temp = temp.reset_index(drop=False)
    temp[time_column] = temp[time_column].astype(float,errors='ignore')
    # temp = temp.sort_values(by = time_column,ascending=True)

    fig = px.bar(
        temp,
        y='Delta',
        x=time_column,
        color='Category',
        hover_data=[time_column,'Category','Delta'],
        template = plotly_theme,
        # text_auto=True,
        text = 'Category',
        height=600,
        )
    
    fig.show()
    


In [48]:

make_barchart(df=df_07days,time_column='YYYYMMDD',title='7 days')
make_barchart(df=df_30days,time_column='YYYY.W',title='30 days')
make_barchart(df=df_90days,time_column='YYYYMM',title='90 Days')
make_barchart(df=df_year,time_column='YYYYMM',title='Year')

In [49]:
# print(*list(df_year.Category.drop_duplicates()),sep='\n')

In [56]:
categories = [
    'health',
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)

In [57]:
categories = [
    'tech'
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)


In [59]:
Locations = [
    'Hershey'
]


mask = df_year['Location'].isin(Locations)
temp = df_year[mask]
title = f'Year - filtered Location ({Locations})'
make_barchart(df=temp,time_column='YYYYMM',title=title)

# 📈 Line-Graph

In [60]:
def make_linegraph(df,title):
    display(HTML('<h2> 📈 Line-Graph: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    df = df.sort_values(by = 'Date',ascending=False)

    fig = px.line(
        df,
        y='Balance',
        x='Date',
        hover_data=['Date','Category','Location','Delta'],
        template = plotly_theme,
        height=600
    )
    fig.show()

In [61]:
make_linegraph(df_07days,'7 days')
make_linegraph(df_30days,'30 days')
make_linegraph(df_90days,'90 days')
make_linegraph(df_year,'Year')

# 📈 Scatter-Graph

In [62]:
def make_scattergraph(df,title):
    display(HTML('<h2> 📈 Scatter-Graph: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))

    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp['size'] = 0.0
    temp['size'] = temp['Delta']
    # temp.loc[temp['size'] >= 1000,'size'] = 1000
    # temp.loc[temp['size'] < 1,'size'] = 1
    

    # display(temp)
    
    fig = px.scatter(
        temp,
        y='Balance',
        x='Date',
        size='size',
        color='Category',
        # color='Delta',
        hover_data=['Date','Category','Location','Delta'],
        template = plotly_theme,
        height=1000,
        size_max=100,
        opacity=0.80,
    )
    
    fig.update_traces(
        marker=dict(
        # size=12,
        line=dict(
            width=1,
            color='white',
            )
        ),
        selector=dict(mode='markers')
    )
    
    fig.show()

In [63]:
make_scattergraph(df_07days,'7 days')
make_scattergraph(df_30days,'30 days')
make_scattergraph(df_90days,'90 days')
make_scattergraph(df_year,'Year')